In [ ]:
import os

import pandas as pd
import sqlalchemy as sa


def env(name: str, default: str) -> str:
    return os.getenv(name, default)


POSTGRES_HOST = env("POSTGRES_HOST", "213.171.31.111")
POSTGRES_PORT = int(env("POSTGRES_PORT", "5433"))
POSTGRES_DB = env("POSTGRES_DB", "blockchain")
POSTGRES_USER = env("POSTGRES_USER", "postgres")
POSTGRES_PASSWORD = env("POSTGRES_PASSWORD", "postgres")

engine = sa.create_engine(
    f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)


def to_regclass(qualified_name: str) -> str | None:
    """Return table name if exists, else None."""
    q = sa.text("select to_regclass(:name) as reg")
    with engine.connect() as conn:
        return conn.execute(q, {"name": qualified_name}).scalar()


def resolve_table(*candidates: str) -> str:
    for c in candidates:
        if to_regclass(c):
            return c
    raise RuntimeError(f"Table not found. Tried: {candidates}")


def read_sql(query: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(sa.text(query), conn)


DAILY_TBL = resolve_table(
    "analytics.daily_transaction_summary",
    "daily_transaction_summary",
)
WALLET_FACT_TBL = resolve_table(
    "analytics.fct_wallet_activity",
    "fct_wallet_activity",
)
WALLET_DIM_TBL = resolve_table(
    "analytics.dim_wallets",
    "dim_wallets",
)

DAILY_TBL, WALLET_FACT_TBL, WALLET_DIM_TBL


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

# --- Daily KPI timeseries ---
daily = read_sql(
    f"""
    select
        transaction_date,
        transaction_count,
        unique_wallets,
        total_volume_eth,
        failure_rate_pct,
        avg_transaction_value_eth,
        failed_transactions,
        successful_transactions
    from {DAILY_TBL}
    order by transaction_date
    """
)

daily["transaction_date"] = pd.to_datetime(daily["transaction_date"])
daily.tail(5)


In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

sns.lineplot(data=daily, x="transaction_date", y="transaction_count", ax=ax[0])
ax[0].set_title("Daily transactions")

sns.lineplot(data=daily, x="transaction_date", y="total_volume_eth", ax=ax[1])
ax[1].set_title("Daily total volume (ETH)")

sns.lineplot(data=daily, x="transaction_date", y="failure_rate_pct", ax=ax[2])
ax[2].set_title("Failure rate (%)")
ax[2].set_xlabel("date")

plt.tight_layout()
plt.show()

# A few quick numeric insights
if len(daily) > 0:
    last_14 = daily.tail(14)
    insight = {
        "days_covered": int((daily["transaction_date"].max() - daily["transaction_date"].min()).days) + 1,
        "last_day_tx": int(daily.iloc[-1]["transaction_count"]),
        "last_day_volume_eth": float(daily.iloc[-1]["total_volume_eth"]),
        "avg_failure_rate_last_14d_pct": float(last_14["failure_rate_pct"].mean()),
    }
    insight


In [ ]:
# --- Wallet leaderboard ---
wallets = read_sql(
    f"""
    select
        wallet_address,
        actual_tx_count,
        total_volume_eth,
        net_balance_eth,
        first_transaction_at,
        last_transaction_at,
        is_active
    from {WALLET_FACT_TBL}
    """
)

wallets["first_transaction_at"] = pd.to_datetime(wallets["first_transaction_at"], errors="coerce")
wallets["last_transaction_at"] = pd.to_datetime(wallets["last_transaction_at"], errors="coerce")

wallets = wallets.fillna({"total_volume_eth": 0, "actual_tx_count": 0, "net_balance_eth": 0})

top_n = 10
leader = wallets.sort_values("total_volume_eth", ascending=False).head(top_n)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
sns.barplot(data=leader, x="total_volume_eth", y="wallet_address", ax=ax)
ax.set_title(f"Top {top_n} wallets by total volume (ETH)")
ax.set_xlabel("total_volume_eth")
ax.set_ylabel("wallet_address")
plt.tight_layout()
plt.show()

# concentration metric: share of volume accounted by top N
if wallets["total_volume_eth"].sum() > 0:
    share_top_n = leader["total_volume_eth"].sum() / wallets["total_volume_eth"].sum()
    {"top_n": top_n, "top_n_volume_share": float(share_top_n)}


In [ ]:
# --- Active wallets share + net balance distribution ---
dim = read_sql(
    f"""
    select
        wallet_address,
        transaction_count as expected_tx_count,
        actual_tx_count,
        net_balance_eth,
        is_active
    from {WALLET_DIM_TBL}
    """
)

active_share = None
if len(dim) > 0:
    active_share = float(dim["is_active"].fillna(False).mean())

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(data=dim, x="net_balance_eth", bins=30, ax=ax[0])
ax[0].set_title("Net balance distribution (ETH)")

sns.scatterplot(data=dim, x="expected_tx_count", y="actual_tx_count", alpha=0.6, ax=ax[1])
ax[1].set_title("Expected vs actual tx count")
ax[1].set_xlabel("expected_tx_count (from wallets)")
ax[1].set_ylabel("actual_tx_count (from transactions)")

plt.tight_layout()
plt.show()

{"active_wallet_share": active_share}
